In [1]:
import requests
import pandas as pd
import urllib.parse
import time
import os

In [2]:
states=['Maine','Massachusetts','Connecticut','New Hampshire','Rhode Island','Vermont']
hdir=""

In [2]:
# #get the list of poitns within each state
import h5pyd
import numpy as np
import pandas as pd
f = h5pyd.File("/nrel/nsrdb/v3/nsrdb_2012.h5", 'r')
meta = pd.DataFrame(f['meta'][...])
states_b=[b'Maine',b'Massachusetts',b'Connecticut',b'New Hampshire',b'Rhode Island',b'Vermont']
states=['Maine','Massachusetts','Connecticut','New Hampshire','Rhode Island','Vermont']
for ist in range(len(states_b)):
    data = meta.loc[meta['state'] == states_b[ist]] # Note .h5 saves strings as bit-strings
    latlon=data[['latitude','longitude']]
    latlon.to_csv(f'{states[ist]}_lonlat.csv')

In [6]:
def get_response_json_and_handle_errors(response: requests.Response) -> dict:
    if response.status_code != 200:
        print(f"An error has occurred with the server or the request. The request response code/status: {response.status_code} {response.reason}")
        print(f"The response body: {response.text}")
        exit(1)
    try:
        response_json = response.json()
    except:
        print(f"The response couldn't be parsed as JSON, likely an issue with the server, here is the text: {response.text}")
        exit(1)
    if len(response_json['errors']) > 0:
        errors = '\n'.join(response_json['errors'])
        print(f"The request errored out, here are the errors: {errors}")
        exit(1)
    return response_json

API_KEY = ""
EMAIL = ""
BASE_URL = "https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.json?"

In [7]:
#put the points together
npoint_per_request=50
indexes=[]
for state in states:
    df=pd.read_csv(f'./points_NSRDB/{state}_lonlat.csv',index_col=0).index.to_list()
    indexes.extend(df)
npointst=len(indexes) 
nstack=npointst//npoint_per_request+1 
print(nstack)

10208


In [8]:
#put the points together
npoint_per_request=50
indexesa=pd.DataFrame()
for state in states:
    dfa=pd.read_csv(f'./points_NSRDB/{state}_lonlat.csv',index_col=0)
    indexesa=pd.concat([indexesa,dfa])


In [9]:
input_data_para = {
        'attributes': 'dhi,dni,ghi,dew_point,wind_speed,air_temperature,surface_pressure',
        'interval': '60',
        'to_utc': 'false',
        'half_hour': 'false',
        'include_leap_day': 'true',
        'api_key': API_KEY,
        'email':'liying_95@126.com'
    }

#download data

In [ ]:
urls=[]
npointst=len(indexes) 
nstack=npointst//npoint_per_request+1 
sy=2007
ey=2007
for yr in range(sy,ey+1):
    print(f"Processing name: {str(yr)}")
    for id in range(0,nstack):
        #aPOINTS = indexes[0:1] #this is one point for testing
        if id!=nstack-1:
            aPOINTS = indexes[id*npoint_per_request:(id+1)*npoint_per_request]
        else:
            aPOINTS = indexes[id*npoint_per_request:]
        POINTS = ', '.join(str(num) for num in aPOINTS)
        print(id)
        input_data=input_data_para.copy()
        input_data['names'] = [str(yr)]
        input_data['location_ids'] = POINTS
        #print(f'Making request for point group {id + 1} of {len(POINTS)}...')

        if '.csv' in BASE_URL:
            url = BASE_URL + urllib.parse.urlencode(data, True)
            # Note: CSV format is only supported for single point requests
            data = pd.read_csv(url)
            #print(f'Response data (you should replace this print statement with your processing): {data}')
            # You can use the following code to write it to a file
            # data.to_csv('SingleBigDataPoint.csv')
        else:
            headers = {'x-api-key': API_KEY}
            data = get_response_json_and_handle_errors(requests.post(BASE_URL, input_data, headers=headers))
            download_url = data['outputs']['downloadUrl']
            # You can do with what you will the download url
            #print(data['outputs']['message'])
            #print(f"Data can be downloaded from this url when ready: {download_url}")

            # Delay for 3 second to prevent rate limiting
            time.sleep(3)
        #print(f'Processed')
        urls.append(download_url)
        with open(f'./urls/urls_{yr}.txt', 'a') as f:
            f.write(download_url + '\n')
        if id!=0:
            if id%19==0:
                time.sleep(60)

#check if all files are downloaded

In [21]:
import glob
yr=2017
leftlist=[]
for index in indexes:
    filename=f'{hdir}/Meteorological_Data/High/Solar/{yr}/{index}*.csv'
    matching_files = glob.glob(filename)
    if not matching_files:
        leftlist.append(index)
print(len(leftlist))
npointst=len(leftlist) 
nstack=npointst//npoint_per_request+1 

11953


In [ ]:
urls=[]
npointst=len(leftlist) 
nstack=npointst//npoint_per_request+1 
for id in range(0,nstack):
    if id!=nstack-1:
        aPOINTS = leftlist[id*npoint_per_request:(id+1)*npoint_per_request]
    else:
        aPOINTS = leftlist[id*npoint_per_request:]
    POINTS = ', '.join(str(num) for num in aPOINTS)
    input_data=input_data_para.copy()
    input_data['names'] = [str(yr)]
    input_data['location_ids'] = POINTS
    if '.csv' in BASE_URL:
        url = BASE_URL + urllib.parse.urlencode(data, True)
        data = pd.read_csv(url)
    else:
        headers = {'x-api-key': API_KEY}
        data = get_response_json_and_handle_errors(requests.post(BASE_URL, input_data, headers=headers))
        download_url = data['outputs']['downloadUrl']
        # Delay for 1 second to prevent rate limiting
        time.sleep(3)
    urls.append(download_url)
    with open(f'./urls/left_{yr}.txt', 'a') as f:
        for item in urls:
            f.write(item + '\n')  # Write the item and move to the next line    
    if id!=0:
        if id%19==0:
            time.sleep(60)